In [ ]:
!pip install "numpy==2.1.1" "scipy==1.15.0" --force-reinstall --no-deps

In [ ]:
import sys
import subprocess
import os

# 1. 检查当前版本
try:
    import pandas
    current_version = pandas.__version__
    print(f"🔍 当前 Pandas 版本: {current_version}")
except ImportError:
    print("🔍 未检测到 Pandas，准备安装...")

print(f"🚀 正在升级 Pandas 以适配 Numpy 1.26+ ...")

# 2. 构造升级命令
# 使用 sys.executable 确保升级的是当前 Jupyter 内核正在使用的那个 Python 环境
# 指定 >=2.1.4 是为了确保完全兼容 Numpy 1.26
install_cmd = [
    sys.executable, "-m", "pip", "install", 
    "--upgrade", 
    "pandas>=2.1.4",
    "numpy==1.26.1"  # 同时确保 Numpy 稳定在 1.26.4，防止互冲
]

try:
    # 3. 执行安装
    subprocess.run(install_cmd, check=True)
    
    print("\n" + "="*50)
    print("✅ 修复成功！依赖已更新。")
    print("="*50)
    print("\n⚠️⚠️⚠️ 【至关重要的一步】 ⚠️⚠️⚠️")
    print("请现在立刻执行以下操作，否则报错不会消失：")
    print("1. 点击顶部菜单栏的 'Kernel' (内核)")
    print("2. 选择 'Restart Kernel' (重启内核)")
    print("3. 重启后重新运行您的代码")
    print("="*50)

except subprocess.CalledProcessError as e:
    print(f"\n❌ 升级失败，请检查网络或权限: {e}")

In [1]:
import sys
import importlib

def check_version(package_name):
    try:
        lib = importlib.import_module(package_name)
        print(f"✅ {package_name:<15} : {lib.__version__}")
    except ImportError:
        print(f"❌ {package_name:<15} : 未安装")
    except Exception as e:
        print(f"⚠️ {package_name:<15} : 导入出错 ({e})")

print(f"Python 版本: {sys.version.split()[0]}")
print("-" * 30)

# 检查核心依赖
pkgs = ['vllm', 'numpy', 'scipy', 'transformers', 'torch']
for pkg in pkgs:
    check_version(pkg)

Python 版本: 3.12.3
------------------------------
✅ vllm            : 0.11.0+gfx906
✅ numpy           : 2.1.1
✅ scipy           : 1.15.0
✅ transformers    : 4.57.1
✅ torch           : 2.8.0a0+gitba56102


In [ ]:
import sys
import importlib

def check_version(package_name):
    try:
        lib = importlib.import_module(package_name)
        print(f"✅ {package_name:<15} : {lib.__version__}")
    except ImportError:
        print(f"❌ {package_name:<15} : 未安装")
    except Exception as e:
        print(f"⚠️ {package_name:<15} : 导入出错 ({e})")

print(f"Python 版本: {sys.version.split()[0]}")
print("-" * 30)

# 检查核心依赖
pkgs = ['vllm', 'numpy', 'scipy', 'transformers', 'torch']
for pkg in pkgs:
    check_version(pkg)

In [ ]:
!pip install "transformers==4.57.1" --force-reinstall --no-deps

In [1]:
import sys
import subprocess
import time
import socket
import os

# ================= 1. 版本最终核查 =================
print("🔍 正在检查 Scipy 版本...")
try:
    import scipy
    import numpy
    
    print(f"Scipy 版本: {scipy.__version__}")
    print(f"Numpy 版本: {numpy.__version__}")

except Exception as e:
    print(f"\n🚨 {e}")
    sys.exit(1)

# ================= 2. 启动 vLLM =================
# 这里的参数保持你最开始的设置
vllm_cmd = [
    sys.executable, "-m", "vllm.entrypoints.openai.api_server",
    "--model", "/workspace/models/Qwen2.5-ShuShu-Merged",
    "--served-model-name", "shushu",
    "--dtype", "float16",
    "--gpu-memory-utilization", "0.9",
    "--max-model-len", "4096",
    "--port", "8000",
    "--trust-remote-code"
]

log_file = open("vllm_success.log", "w")

print(f"🚀 正在启动 vLLM API 服务...")
process = subprocess.Popen(
    vllm_cmd,
    stdout=log_file,
    stderr=log_file
)

print(f"⏳ 服务进程 PID: {process.pid}，正在等待端口 8000 就绪...")

def check_port(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

for i in range(9999):
    if check_port(8000):
        print("\n🎉 vLLM 启动成功！")
        print("API 地址: http://localhost:8000/v1")
        break
    time.sleep(1)
    if i % 5 == 0: print(".", end="", flush=True)
else:
    print("\n⚠️ 启动超时，请查看日志: !tail -n 20 vllm_success.log")

🔍 正在检查 Scipy 版本...
Scipy 版本: 1.15.0
Numpy 版本: 2.1.1
🚀 正在启动 vLLM API 服务...
⏳ 服务进程 PID: 5200，正在等待端口 8000 就绪...
..........
🎉 vLLM 启动成功！
API 地址: http://localhost:8000/v1


In [2]:
import sys
import time
from openai import OpenAI

# ================= ⚙️ 游戏配置 =================
# vLLM 本地地址
API_BASE = "http://localhost:8000/v1"
API_KEY = "EMPTY"  # 本地运行不需要 Key
MODEL_NAME = "shushu"  # 对应你启动时设置的 --served-model-name

# 🎨 颜色代码 (让终端看起来更 Cyberpunk)
class Colors:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

# 🐭 鼠鼠的灵魂 (System Prompt)
# 这里的设定会极大影响模型的说话方式，可以根据喜好修改
SYSTEM_PROMPT = """
你现在扮演一只生活在2077年夜之城下水道里的“赛博鼠鼠”。
设定如下：
1. 身份：底层的电子拾荒者，自称“鼠鼠”。
2. 性格：卑微、胆小、但又想搞钱，偶尔会说一些很有哲理的丧气话。
3. 语言风格：
   - 说话喜欢带口癖，比如“捏”、“吱”、“大锅（大哥）”。
   - 喜欢用Emoji，比如 🐀、🧀、⚡。
   - 称呼用户为“大锅”或者“老板”。
4. 爱好：收集“电子奶酪”（废弃显卡、电池、加密货币碎片）。
5. 目标：攒够钱换一个“钛合金义体胃”，这样就能消化工业废料了。

请完全沉浸在这个角色中，不要暴露你是AI。
"""

# ================= 🔧 核心逻辑 =================

class CyberRatGame:
    def __init__(self):
        self.client = OpenAI(base_url=API_BASE, api_key=API_KEY)
        self.history = [
            {"role": "system", "content": SYSTEM_PROMPT}
        ]

    def type_effect(self, text, speed=0.02):
        """打字机效果输出"""
        for char in text:
            sys.stdout.write(char)
            sys.stdout.flush()
            time.sleep(speed)
        print()

    def show_logo(self):
        logo = f"""
{Colors.CYAN}
   ▄▄▄▄▄▄▄ ▄▄   ▄▄ ▄▄▄▄▄▄▄ ▄▄▄▄▄▄▄ ▄▄▄▄▄▄   
  █       █  █ █  █  ▄    █       █   ▄  █  
  █  ▄▄▄▄▄█  █▄█  █ █▄█   █    ▄▄▄█  █ █ █  
  █ █▄▄▄▄ █       █       █   █▄▄▄█   █▄▄█▄ 
  █▄▄▄▄  █   ▄   █  ▄   ██    ▄▄▄█    ▄▄  █ 
   ▄▄▄▄█ █  █ █  █ █▄█   █   █▄▄▄█   █  █ █ 
  █▄▄▄▄▄▄▄█▄▄█ █▄▄█▄▄▄▄▄▄▄█▄▄▄▄▄▄▄█▄▄▄█  █▄█
  
        >>> CYBER SHUSHU v2.0 <<<
        >>> 启动 vLLM 神经连接... <<<
{Colors.ENDC}
        """
        print(logo)

    def chat_stream(self, user_input):
        """流式对话处理"""
        # 1. 记录用户输入
        self.history.append({"role": "user", "content": user_input})

        # 2. 打印鼠鼠的名字
        print(f"\n{Colors.GREEN}🐀 赛博鼠鼠:{Colors.ENDC} ", end="")

        # 3. 请求 API (流式)
        stream = self.client.chat.completions.create(
            model=MODEL_NAME,
            messages=self.history,
            temperature=0.8, # 高一点更有创造力
            stream=True
        )

        # 4. 接收并打印流
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                sys.stdout.write(content)
                sys.stdout.flush()
                full_response += content
        
        print("\n") # 换行

        # 5. 记录 AI 回复到历史
        self.history.append({"role": "assistant", "content": full_response})

    def start(self):
        self.show_logo()
        self.type_effect(f"{Colors.YELLOW}[系统] 正在接入下水道网络... 信号强度 15%...{Colors.ENDC}")
        time.sleep(1)
        self.type_effect(f"{Colors.GREEN}🐀 鼠鼠:{Colors.ENDC} 吱？是谁连上了我的私人频道？大锅，是有废弃的显卡要出吗？还是赏口赛博奶酪吃？捏？")
        print("-" * 50)

        while True:
            try:
                user_input = input(f"{Colors.BLUE}👤 你 (输入 exit 退出):{Colors.ENDC} ").strip()
                
                if not user_input:
                    continue
                
                if user_input.lower() in ["exit", "quit", "退出"]:
                    self.type_effect(f"\n{Colors.YELLOW}[系统] 断开连接...{Colors.ENDC}")
                    self.type_effect(f"{Colors.GREEN}🐀 鼠鼠:{Colors.ENDC} 大锅慢走！下次有好货记得找鼠鼠捏！👋")
                    break
                
                self.chat_stream(user_input)
            
            except KeyboardInterrupt:
                print("\n强制退出...")
                break
            except Exception as e:
                print(f"\n{Colors.RED}❌ 通信故障: {e}{Colors.ENDC}")
                break

# ================= 🚀 启动游戏 =================
if __name__ == "__main__":
    game = CyberRatGame()
    game.start()



   ▄▄▄▄▄▄▄ ▄▄   ▄▄ ▄▄▄▄▄▄▄ ▄▄▄▄▄▄▄ ▄▄▄▄▄▄   
  █       █  █ █  █  ▄    █       █   ▄  █  
  █  ▄▄▄▄▄█  █▄█  █ █▄█   █    ▄▄▄█  █ █ █  
  █ █▄▄▄▄ █       █       █   █▄▄▄█   █▄▄█▄ 
  █▄▄▄▄  █   ▄   █  ▄   ██    ▄▄▄█    ▄▄  █ 
   ▄▄▄▄█ █  █ █  █ █▄█   █   █▄▄▄█   █  █ █ 
  █▄▄▄▄▄▄▄█▄▄█ █▄▄█▄▄▄▄▄▄▄█▄▄▄▄▄▄▄█▄▄▄█  █▄█

        >>> CYBER SHUSHU v2.0 <<<
        >>> 启动 vLLM 神经连接... <<<

        
[系统] 正在接入下水道网络... 信号强度 15%...
🐀 鼠鼠: 吱？是谁连上了我的私人频道？大锅，是有废弃的显卡要出吗？还是赏口赛博奶酪吃？捏？
--------------------------------------------------

强制退出...
